In [1]:
import cartopy

In [20]:
import xarray as xr
import sys
sys.path.insert(0,"/home/albert/lib/python")

from netCDF4 import Dataset

import numpy as np
sys.path.insert(0,"/home/albert/Work/git/xscale")
import xscale
import xscale.spectral.fft as xfft
from matplotlib.colors import LogNorm
import cmocean
import matplotlib.pylab as plt
import glob
from datetime import date

import cartopy.crs as ccrs
import cartopy.feature as cfeature
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER

In [29]:
fileUtide="/mnt/albert/equipes/IGE/meom/MODEL_SET/eNATL60/eNATL60-BLBT02-S/1h/ACO/eNATL60ACO-BLBT02_1h_20090710_20090809_gridU-2D.nc"
fileVtide="/mnt/albert/equipes/IGE/meom/MODEL_SET/eNATL60/eNATL60-BLBT02-S/1h/ACO/eNATL60ACO-BLBT02_1h_20090710_20090809_gridV-2D.nc"

fileUnotide="/mnt/albert/equipes/IGE/meom/MODEL_SET/eNATL60/eNATL60-BLB002-S/1h/ACO/eNATL60ACO-BLB002_1h_20090710_20090809_gridU-2D.nc"
fileVnotide="/mnt/albert/equipes/IGE/meom/MODEL_SET/eNATL60/eNATL60-BLB002-S/1h/ACO/eNATL60ACO-BLB002_1h_20090710_20090809_gridV-2D.nc"


In [31]:
fUtide = xr.open_mfdataset(fileUtide,chunks={'x':500,'y':500})
Utide=fUtide['sozocrtx'][:]
navlat= fUtide['nav_lat'][0]
navlon= fUtide['nav_lon'][0]

fUnotide = xr.open_mfdataset(fileUnotide,chunks={'x':500,'y':500})
Unotide=fUnotide['sozocrtx'][:]

fVtide = xr.open_mfdataset(fileVtide,chunks={'x':500,'y':500})
Vtide=fVtide['somecrty'][:]

fVnotide = xr.open_mfdataset(fileVnotide,chunks={'x':500,'y':500})
Vnotide=fVnotide['somecrty'][:]


In [32]:
T=2*np.pi/(1E-4)

In [33]:
wutide=Utide.window
wutide.set(n=30,dim='time_counter', cutoff=T)
Utidefilt=wutide.convolve()

In [34]:
wunotide=Unotide.window
wunotide.set(n=30,dim='time_counter', cutoff=T)
Unotidefilt=wunotide.convolve()

In [35]:
wvtide=Vtide.window
wvtide.set(n=30,dim='time_counter', cutoff=T)
Vtidefilt=wvtide.convolve()

In [36]:
wvnotide=Vnotide.window
wvnotide.set(n=30,dim='time_counter', cutoff=T)
Vnotidefilt=wvnotide.convolve()

In [38]:
in_Utide_filt=fileUtide
dstin=Dataset(in_Utide_filt,'r')
out_Utide_filt='eNATL60ACO-BLBT02_1h_20090710_20090809_gridU-2D_filt.nc'
dstout=Dataset(dirfilttide+out_Utide_filt,'w')

varname='sozocrtx'

today=date.today()
dstout.description = "Data time filtered with Lanczos filter with window size of "+str(30)+" and cut-off frequency of "+str(T)+" obtained "+str(today.day)+"/"+str(today.month)+"/"+str(today.year)

for dname, the_dim in dstin.dimensions.iteritems():
    dstout.createDimension(dname, len(the_dim) if not the_dim.isunlimited() else None)

for v_name, varin in dstin.variables.iteritems():
    if v_name == varname:
        continue
    outVar = dstout.createVariable(v_name, varin.datatype, varin.dimensions)
    outVar.setncatts({k: varin.getncattr(k) for k in varin.ncattrs()})
    outVar[:] = varin[:]

datain=dstin[varname]
dataout=dstout.createVariable(varname+'_filt',datain.datatype,datain.dimensions)
dataout.setncatts({k: datain.getncattr(k) for k in datain.ncattrs()})

ztemp=np.zeros(datain.shape)
ztemp[:]=Utidefilt[:]
    
dataout[:]=ztemp[:]
dstout.close()

In [39]:
in_Vtide_filt=fileVtide
dstin=Dataset(in_Vtide_filt,'r')
out_Vtide_filt='eNATL60ACO-BLBT02_1h_20090710_20090809_gridV-2D_filt.nc'
dstout=Dataset(dirfilttide+out_Vtide_filt,'w')

varname='somecrty'

today=date.today()
dstout.description = "Data time filtered with Lanczos filter with window size of "+str(30)+" and cut-off frequency of "+str(T)+" obtained "+str(today.day)+"/"+str(today.month)+"/"+str(today.year)

for dname, the_dim in dstin.dimensions.iteritems():
    dstout.createDimension(dname, len(the_dim) if not the_dim.isunlimited() else None)

for v_name, varin in dstin.variables.iteritems():
    if v_name == varname:
        continue
    outVar = dstout.createVariable(v_name, varin.datatype, varin.dimensions)
    outVar.setncatts({k: varin.getncattr(k) for k in varin.ncattrs()})
    outVar[:] = varin[:]

datain=dstin[varname]
dataout=dstout.createVariable(varname+'_filt',datain.datatype,datain.dimensions)
dataout.setncatts({k: datain.getncattr(k) for k in datain.ncattrs()})

ztemp=np.zeros(datain.shape)
ztemp[:]=Vtidefilt[:]
    
dataout[:]=ztemp[:]
dstout.close()

In [40]:
in_Unotide_filt=fileUnotide
dstin=Dataset(in_Unotide_filt,'r')
out_Unotide_filt='eNATL60ACO-BLB002_1h_20090710_20090809_gridU-2D_filt.nc'
dstout=Dataset(dirfiltnotide+out_Unotide_filt,'w')

varname='sozocrtx'

today=date.today()
dstout.description = "Data time filtered with Lanczos filter with window size of "+str(30)+" and cut-off frequency of "+str(T)+" obtained "+str(today.day)+"/"+str(today.month)+"/"+str(today.year)

for dname, the_dim in dstin.dimensions.iteritems():
    dstout.createDimension(dname, len(the_dim) if not the_dim.isunlimited() else None)

for v_name, varin in dstin.variables.iteritems():
    if v_name == varname:
        continue
    outVar = dstout.createVariable(v_name, varin.datatype, varin.dimensions)
    outVar.setncatts({k: varin.getncattr(k) for k in varin.ncattrs()})
    outVar[:] = varin[:]

datain=dstin[varname]
dataout=dstout.createVariable(varname+'_filt',datain.datatype,datain.dimensions)
dataout.setncatts({k: datain.getncattr(k) for k in datain.ncattrs()})

ztemp=np.zeros(datain.shape)
ztemp[:]=Unotidefilt[:]
    
dataout[:]=ztemp[:]
dstout.close()

In [41]:
in_Vnotide_filt=fileVnotide
dstin=Dataset(in_Vnotide_filt,'r')
out_Vnotide_filt='eNATL60ACO-BLB002_1h_20090710_20090809_gridV-2D_filt.nc'
dstout=Dataset(dirfiltnotide+out_Vnotide_filt,'w')

varname='somecrty'

today=date.today()
dstout.description = "Data time filtered with Lanczos filter with window size of "+str(30)+" and cut-off frequency of "+str(T)+" obtained "+str(today.day)+"/"+str(today.month)+"/"+str(today.year)

for dname, the_dim in dstin.dimensions.iteritems():
    dstout.createDimension(dname, len(the_dim) if not the_dim.isunlimited() else None)

for v_name, varin in dstin.variables.iteritems():
    if v_name == varname:
        continue
    outVar = dstout.createVariable(v_name, varin.datatype, varin.dimensions)
    outVar.setncatts({k: varin.getncattr(k) for k in varin.ncattrs()})
    outVar[:] = varin[:]

datain=dstin[varname]
dataout=dstout.createVariable(varname+'_filt',datain.datatype,datain.dimensions)
dataout.setncatts({k: datain.getncattr(k) for k in datain.ncattrs()})

ztemp=np.zeros(datain.shape)
ztemp[:]=Vnotidefilt[:]
    
dataout[:]=ztemp[:]
dstout.close()